# Parte B: Evaluación de modelos de AA

In [8]:
  
from surprise import Dataset, Reader, KNNBasic, SVD, NMF
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
# Definir SEED
SEED = 42

## 1


In [10]:
# Cargar el dataset de MovieLens de 100K
data = Dataset.load_builtin('ml-100k')

## 2

In [11]:
# Dividir el dataset
trainset, testset = train_test_split(data, test_size=0.25, random_state=SEED)

## 3

In [12]:
# Evaluar distintos algoritmos de recomendación
# Filtrado colaborativo basado en vecinos (KNNBasic)
algo_knn_user = KNNBasic(sim_options={'name': 'pearson', 'user_based': True}, random_state=SEED)
algo_knn_item = KNNBasic(sim_options={'name': 'pearson', 'user_based': False}, random_state=SEED)

In [13]:
# Filtrado colaborativo basado en modelos (SVD y NMF)
algo_svd = SVD(random_state=SEED)
algo_nmf = NMF(random_state=SEED)

## 4 y 5

In [14]:
# Entrenar y evaluar los algoritmos
algorithms = [("KNN-User-Based", algo_knn_user),
              ("KNN-Item-Based", algo_knn_item),
              ("SVD", algo_svd),
              ("NMF", algo_nmf)]




In [15]:
# Mostrar resultados de 5 predicciones para cada algoritmo
for name, algo in algorithms:
    print(f"Evaluando {name}...")
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions)
    print(f"{name} RMSE: {rmse}")

    print("Algunas predicciones:")
    for pred in predictions[:5]:
        print(pred)
    print()

Evaluando KNN-User-Based...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0182
KNN-User-Based RMSE: 1.018246371164168
Algunas predicciones:
user: 391        item: 591        r_ui = 4.00   est = 3.69   {'actual_k': 40, 'was_impossible': False}
user: 181        item: 1291       r_ui = 1.00   est = 3.08   {'actual_k': 6, 'was_impossible': False}
user: 637        item: 268        r_ui = 2.00   est = 3.58   {'actual_k': 40, 'was_impossible': False}
user: 332        item: 451        r_ui = 5.00   est = 3.17   {'actual_k': 40, 'was_impossible': False}
user: 271        item: 204        r_ui = 4.00   est = 3.82   {'actual_k': 40, 'was_impossible': False}

Evaluando KNN-Item-Based...
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0465
KNN-Item-Based RMSE: 1.046509372244098
Algunas predicciones:
user: 391        item: 591        r_ui = 4.00   est = 3.72   {'actual_k': 40, 'was_impossible': False}
user: 181        item: 1

Fijándonos solo en el usuario 391 el cual valoro la película con un 4.0, el modelo KNN-User estimo que valoraría la película xon un 3.69, el modelo KNN-Item estimo una puntuacion de 3.72, el modelo SVD estimo una puntuación de 3.52 y el modelo NMF estimo una puntuación de 3.7
## SOLO ESTABA COMPARANDO LAS ESTIMACIONES A UN USUARIO CONCRETO ASI QUE CUANDO ANALICEMOS LOS DATOS AL COMPLETO SACAREMOS CONCLUSIONES

## 6

Creamos una función para obtener las metricas Precision@10, Recall@10 y NDCG@10

In [18]:
from sklearn.metrics import precision_score, recall_score, ndcg_score
import numpy as np
def evaluate_metrics(predictions, k=10):
    # Convertir predicciones a un formato manejable
    relevant_items = []
    recommended_items = []
    scores = []

    for uid, iid, true_r, est, details in predictions:
        relevant_items.append(1 if true_r > 4 else 0)  # 1: relevante, 0: no relevante
        scores.append(est)

    # Obtener índices de las top-k recomendaciones
    sorted_indices = np.argsort(scores)[::-1][:k]
    recommended_items = [1 if i in sorted_indices else 0 for i in range(len(scores))]

    # Calcular Precision@k, Recall@k, NDCG@k
    precision = precision_score(relevant_items, recommended_items, zero_division=1)
    recall = recall_score(relevant_items, recommended_items, zero_division=1)
    ndcg = ndcg_score([relevant_items], [scores], k=k)

    return precision, recall, ndcg

evaluamos cada algoritmo

In [19]:
# Evaluar cada algoritmo
results = []
for name, algo in algorithms:
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)
    precision, recall, ndcg = evaluate_metrics(predictions, k=10)

    results.append({
        "Modelo": name,
        "RMSE": rmse,
        "Precision@10": precision,
        "Recall@10": recall,
        "NDCG@10": ndcg
    })
import pandas as pd

Computing the pearson similarity matrix...
Done computing similarity matrix.


ValueError: Found input variables with inconsistent numbers of samples: [25000, 0]

In [11]:
# Crear un DataFrame a partir de los resultados
results_df = pd.DataFrame(results)

In [12]:
# Mostrar la tabla
print(results_df)


           Modelo      RMSE  Precision@10  Recall@10   NDCG@10
0  KNN-User-Based  1.018246           0.7   0.001324  0.571429
1  KNN-Item-Based  1.046509           0.5   0.000946  0.538462
2             SVD  0.939557           0.8   0.001513  0.794521
3             NMF  0.965033           0.6   0.001135  0.686275


# Conclusiones:
- **RMSE**: El modelo **SVD** tiene el menor error (RMSE más bajo), lo que indica que realiza predicciones más cercanas a las calificaciones reales.
- **Precision@10**: El modelo **SVD** proporciona la mayor precisión, es decir, recomienda un mayor número de películas relevantes entre las top-10.
- **Recall@10**: **SVD** también tiene el mejor desempeño al recuperar un mayor porcentaje de películas relevantes.
- **NDCG@10**: **SVD** cuenta con el mayor valor de NDCG, lo que indica una mejor organización de recomendaciones relevantes en posiciones altas de la lista.

El modelo **SVD** es el mejor entre los evaluados porque:
1. Tiene el menor RMSE, lo que confirma la calidad de sus predicciones.
2. Sobresale en las métricas de Precision@10, Recall@10 y NDCG@10, demostrando que ofrece películas relevantes y bien ordenadas dentro de las top-10